In [ ]:
# Subset mutations for Sellers lab
# only include CCLE2 lines

import pandas as pd
import dalmatian as dm

In [ ]:
ccle2_ws = "fccredits-silver-tan-7621/CCLE_v2"

In [ ]:
ccle2wm = dm.WorkspaceManager(ccle2_ws).disable_hound()

In [ ]:
ccle2samples = ccle2wm.get_samples()

In [ ]:
ccle2samples.index

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()
merged_somatic_mutations = tc.get(name='mutations-latest-ed72', version=24, file='merged_somatic_mutations')

In [ ]:
merged_somatic_mutations

In [ ]:
subset_mut = merged_somatic_mutations[merged_somatic_mutations.DepMap_ID.isin(ccle2samples.index)]
subset_mut

In [ ]:
subset_mut.to_csv("ccle2_mutations.csv", index=None)

In [ ]:
merged_segments_all = tc.get(name='cn-latest-d8d4', version=26, file='merged_segments_all')


In [ ]:
merged_segments_all

In [ ]:
GBM_names = ["42MGBA", "8MGBA", "A172", "AM38", "BECKER", "CAS1", "CCFSTTG1", "DBTRG05MG", "DKMG", "GAMG", "GB1", "GI1", "GMS10", "GOS3", "HS683", "KALS1", "KG1C", "KNS42", "KNS60", "KNS81", "LN18", "LN229", "M059K", "NMCG1", "SF126", "SF268", "SF295", "SF539", "SNB75", "SNU1105", "SNU201", "SNU466", "SNU489", "SNU626", "SNU738", "SW1088", "SW1783", "T98G", "TM31", "U118MG", "U251MG", "U87MG", "YH13", "YKG1"]

In [ ]:
from depmapomics import tracker as track
trackerobj = track.initTracker()
t = trackerobj.read_tracker()
t

In [ ]:
GBM_rows = t[t.stripped_cell_line_name.isin(GBM_names)]
GBM_rows

In [ ]:
len(GBM_rows.arxspan_id.unique())

In [ ]:
GBM_seg = merged_segments_all[merged_segments_all.DepMap_ID.isin(GBM_rows.arxspan_id.unique())]
GBM_seg

In [ ]:
GBM_seg.to_csv("ccle_seg.csv", index=None)

In [ ]:
# Check cclf twist snv output
twistwm = dm.WorkspaceManager("nci-mimoun-bi-org/PANCAN_TWIST copy").disable_hound()

In [ ]:
twist_pairsets = twistwm.get_pair_sets()
twist_pairsets

In [ ]:
twist41snv = pd.read_csv(twist_pairsets.loc["CCLF_TWIST41_pairs", "aggregate_snvs"], sep="\t")
twist41snv

In [ ]:
twist41subset = twist41snv[twist41snv.external_id == "ACH-003025_BOM_P7_3D_1"]
twist41subset[twist41subset.Chromosome == "16"]['Hugo_Symbol']

In [ ]:
twist41subset[twist41subset.Chromosome == "16"][['Start_position', 'End_position']]

In [ ]:
twist41snv[twist41snv.external_id == "ACH-003025_BOM_P7_3D_1"].to_csv("ACH003025.csv", index=None)

In [ ]:
maf = pd.read_csv("gs://fc-secure-bd7b8bc9-f665-4269-997e-5a402088a369/1cdc902e-234f-450c-9303-406fb34ff6ad/WGS_pipeline/b6bc83be-af09-4ac2-8c91-57bf070b8870/call-filterMaf/CDS-0JJKl3_outMAFfn.txt", sep="\t")
maf


In [ ]:
maf.dbNSFP_Polyphen2_HVAR_pred.unique()

In [ ]:
proteincoding_genes_tpm_logp1 = tc.get(name='expression-d035', version=21, file='proteincoding_genes_tpm_logp1')
proteincoding_genes_tpm_logp1

In [ ]:
filter_col = [col for col in proteincoding_genes_tpm_logp1 if col.startswith(('ERBB2 (', 'ESR1 (', 'PGR ('))]
filter_col

In [ ]:
from scipy.stats import zscore
zscores = proteincoding_genes_tpm_logp1.apply(zscore)
zscores

In [ ]:
minerva_lines = ["ACH-002949", "ACH-002839", "ACH-002741", "ACH-002885", "ACH-002950", "ACH-002951", "ACH-002954", "ACH-002968", "ACH-002981", "ACH-002883", "ACH-002835", "ACH-002744", "ACH-002837", "ACH-002730", "ACH-002742", "ACH-002743", "ACH-002947", "ACH-002871", "ACH-002735", "ACH-002757", "ACH-002972", "ACH-002869", "ACH-002737", "ACH-002853", "ACH-002738", "ACH-002979", "ACH-002841", "ACH-002736", "ACH-002884", "ACH-002745", "ACH-002967", "ACH-002733"]
minerva_zscores = zscores[zscores.index.isin(minerva_lines)].T
minerva_zscores.to_csv("minerva_expression_zscores.csv")

In [ ]:
minerva_zscores.loc["ACH-002949", "PGR (5241)"]

In [ ]:
from biomart import BiomartServer
import io

def _fetchFromServer(ensemble_server, attributes):
    server = BiomartServer(ensemble_server)
    ensmbl = server.datasets["hsapiens_gene_ensembl"]
    res = pd.read_csv(
        io.StringIO(
            ensmbl.search({"attributes": attributes}, header=1).content.decode()
        ),
        sep="\t",
    )
    return res

def generateGeneNames(
    ensemble_server="http://nov2020.archive.ensembl.org/biomart",
    useCache=False,
    cache_folder="",
    attributes=[],
):
    """generate a genelist dataframe from ensembl's biomart

  Args:
      ensemble_server ([type], optional): [description]. Defaults to ENSEMBL_SERVER_V.
      useCache (bool, optional): [description]. Defaults to False.
      cache_folder ([type], optional): [description]. Defaults to CACHE_PATH.

  Raises:
      ValueError: [description]

  Returns:
      [type]: [description]
  """
    attr = [
        "ensembl_gene_id",
        "hgnc_symbol",
        "gene_biotype",
        "entrezgene_id",
    ]
    #assert cache_folder[-1] == "/"

    cache_folder = os.path.expanduser(cache_folder)
    #createFoldersFor(cache_folder)
    cachefile = os.path.join(cache_folder, ".biomart.csv")
    if useCache & os.path.isfile(cachefile):
        print("fetching gene names from biomart cache")
        res = pd.read_csv(cachefile)
    else:
        print("downloading gene names from biomart")
        res = _fetchFromServer(ensemble_server, attr + attributes)
        res.to_csv(cachefile, index=False)

    res.columns = attr + attributes
    if type(res) is not type(pd.DataFrame()):
        raise ValueError("should be a dataframe")
    res = res[~(res["hgnc_symbol"].isna())]

    return res

In [ ]:
from genepy.utils import helper as h

mybiomart = generateGeneNames(ensemble_server="http://dec2021.archive.ensembl.org/biomart", useCache=False)
mybiomart

In [ ]:
mybiomart[mybiomart.entrezgene_id == 1394.0]›

In [ ]:
protmybiomart = mybiomart[mybiomart.gene_biotype == "protein_coding"]

In [ ]:
protmybiomart

In [ ]:
mybiomart_unique = protmybiomart.drop_duplicates(subset='ensembl_gene_id', keep="first")
mybiomart_unique

In [ ]:
grouped = mybiomart_unique.groupby(['entrezgene_id'])

In [ ]:
dups = []
for name, group in grouped:
    if len(group) > 1 and len(set(group.hgnc_symbol)) > 1:
        print(set(group.hgnc_symbol))
        dups.append((str(int(name)), set(group.hgnc_symbol)))

In [ ]:
dups

In [ ]:
mybiomart[mybiomart.entrezgene_id == 3811.0]

In [ ]:
myoldbiomart = generateGeneNames(ensemble_server=ENSEMBL_SERVER_V, useCache=False)
myoldbiomart

In [ ]:
protmyoldbiomart = myoldbiomart[myoldbiomart.gene_biotype == "protein_coding"]
myoldbiomart_unique = protmyoldbiomart.drop_duplicates(subset='ensembl_gene_id', keep="first")
groupedold = myoldbiomart_unique.groupby(['entrezgene_id'])

dupsold = []
for name, group in groupedold:
    if len(group) > 1 and len(set(group.hgnc_symbol)) > 1:
        print(set(group.hgnc_symbol))
        dupsold.append((str(int(name)), set(group.hgnc_symbol)))

In [ ]:
dupsold

In [ ]:
myoldbiomart[myoldbiomart.entrezgene_id == 3811.0]

In [ ]:
myoldbiomart[myoldbiomart.hgnc_symbol == "CRHR1"]

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

CCLE_expression = tc.get(name='internal-22q1-1778', version=24, file='CCLE_expression')
CCLE_expression

In [ ]:
CCLE_expression[['KIR3DL1 (3811)']]

In [ ]:
rnawm = dm.WorkspaceManager('terra-broad-cancer-prod/CCLE_DepMap_RNAseq')
rnasamples = rnawm.get_samples()
rnasamples

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()
achilles_gene_cn = tc.get(name='cn-achilles-version-06ca', version=70, file='achilles_gene_cn')

In [ ]:
achilles_gene_cn

In [ ]:
all_21Q2_gene_cn = tc.get(name='cn-achilles-version-06ca', version=60, file='all_21Q2_gene_cn')

In [ ]:
all_21Q2_gene_cn

In [ ]:
pd.read_csv("../temp/21Q4v2/merged_somatic_mutations_boolmatrix_other.csv")